In [ ]:
from tile import tile_ortho

In [ ]:
bounds = tile_ortho(
    ortho_path='/home/justine/fmr/ngouja_102021/Ngouja_orthomosaic_102021.tif',
    dest_dir='/home/justine/fmr/ngouja_102021/tiles_5000_no_overlap',
    tuple_tile_size=(5000,5000),
    h_shift=0,
    v_shift=0
)

In [ ]:
#Import libraries
import os
import sys
import yaml
import fiftyone as fo
import fiftyone.utils.geojson as foug
import pandas as pd
from predict import build_predictor, add_predictions_to_dataset
import fiftyone as fo
import fiftyone.utils.coco as fouc
import torch
from config_model import add_config
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

from deploy_dataset import create_or_load_dataset


In [ ]:
importer = fouc.COCODetectionDatasetImporter(
    data_path='/home/justine/fmr/tiles_1500_overlap25/',
    labels_path='/home/justine/fmr/tiles_1500_overlap25/coco/coco_test10_species.json', 
    label_types="segmentations",
    include_id=True,
    use_polylines=True,
    tolerance=2)

dataset.add_importer(
    dataset_importer=importer,
    label_field="ground_truth")

In [ ]:
importer = fouc.COCODetectionDatasetImporter(
    data_path='/home/justine/fmr/tiles_1500_overlap25/',
    labels_path='/home/justine/fmr/tiles_1500_overlap25/coco/coco_val10_species.json', 
    label_types="segmentations",
    include_id=True,
    use_polylines=True,
    tolerance=2)

dataset.add_importer(
    dataset_importer=importer,
    label_field="ground_truth")

In [ ]:
dataset = create_or_load_dataset(
    dataset_name='deepmosaics_testset', 
    dataset_type='unlabeled', 
    images_path='/home/justine/fmr/coco/coco_test10_species/', 
    label_type='segmentations')

dataset.persistent = True

dataset.save()

print(dataset)

In [ ]:
config_name = '../configs/models_training/x101_allsites_species_overlapping25_tiles5000_ITER3000.yml'
session = '20230412_14:23:21x101_allsites_species_overlapping25_tiles1500_ITER3000'
device = ("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
cfg = get_cfg()
cfg = add_config(
    cfg=cfg,
    config_path=config_name,
    device=device,
    train_dataset = ("trainset",),
    test_dataset = ("testset",),
    mode='inference'
    )
predictor = DefaultPredictor(cfg)


In [ ]:
#Load model's classes
model_classes = '../configs/inference/model_classes_species.yml'

with open(model_classes) as f:
    model_classes = yaml.load(f, Loader=yaml.FullLoader)


In [ ]:
dataset = add_predictions_to_dataset(
    dataset=dataset, 
    predictor=predictor, 
    device=device, 
    classes=model_classes['species_classes'],
    predictions_field='species_nms50', 
    nms_threshold=0.5)

In [ ]:
dataset

In [ ]:
from fiftyone import ViewField as F
stage = fo.FilterLabels("species_nms50", F("points").length() > 0)
view = dataset.add_stage(stage)

In [ ]:
view

In [ ]:
from fiftyone import ViewField as F
conf_filter = fo.FilterLabels("species_nms50", F("confidence") > 0.3)
view2 = view.add_stage(conf_filter)

In [ ]:
print(view2)

In [ ]:
results = dataset.evaluate_detections(
        pred_field='species_nms50',
        gt_field="ground_truth_segmentations",
        eval_key='eval_predictions_tiles1500',
        method='coco',
        classes=model_classes,
        classwise=False,
        iou=0.50,
        compute_mAP=True)

In [ ]:
cm = results.plot_confusion_matrix(title='Matrice de confusion : evaluation de MODEL_SPECIES_X101_ITER20000')
cm.show()

In [ ]:
coco_exporter = fouc.COCODetectionDatasetExporter(
    export_dir='/home/justine/fmr/tiles_1500_overlap25/coco/',
    labels_path= 'predictions_testset_conf30.json',
    export_media=False,
    classes=model_classes['species_classes'],
    tolerance=0, 
    extra_attrs=False)

view2.export(
    dataset_exporter=coco_exporter,
    label_field='species_nms50')

### spatialize

In [ ]:
from spatialize_coco import coco2gdf, get_transform, affine_transform, export

In [ ]:
gdf = coco2gdf('/home/justine/fmr/tiles_1500_overlap25/coco/predictions_testset_conf30.json')

In [ ]:
gdf['transform'] =  [get_transform('/home/justine/fmr/tiles_1500_overlap25/', sample) for sample in gdf['img_filename']]

In [ ]:
gdf_affined = affine_transform(gdf, 'EPSG:4326')

In [ ]:
export(gdf_affined, '/home/justine/fmr/tiles_1500_overlap25/coco/', 'predictions_testset_conf30', site_by_site=False)